# Fine-Tuning Llama2 with Amazon Bedrock
_Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 3.0) kernel with the ml.t3.medium kernel._

-----------

### 1. Setup
### 2. Test the base model
### 3. Prepare the dataset for fine-tuning
### 4. Upload the dataset to S3
### 5. Customize the model with fine-tuning
### 6. Provision the custom model for inference
### 7. Test the custom model
### 8. Delete the provisioned model to save cost

---

# 1. Setup

In [2]:
%pip install -q -U --force-reinstall \
    pandas==2.1.2 \
    datasets==2.15.0

In [3]:
import boto3
import json
import time
from pprint import pprint
from IPython.display import display, HTML
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 2)
pd.set_option('display.max_colwidth', 1000)

In [4]:
# Amazon Bedrock control plane including fine-tuning
bedrock = boto3.client(service_name="bedrock")

# Amazon Bedrock data plane including model inference
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [5]:
for model in bedrock.list_foundation_models(byProvider="meta", 
                                            byCustomizationType="FINE_TUNING")["modelSummaries"]:
    print("-----\n" + "modelArn: " + model["modelArn"] + "\nmodelId: " + model["modelId"] + "\nmodelName: " + model["modelName"] + "\ncustomizationsSupported: " + ','.join(model["customizationsSupported"]))

-----
modelArn: arn:aws:bedrock:us-east-1::foundation-model/meta.llama2-13b-v1:0:4k
modelId: meta.llama2-13b-v1:0:4k
modelName: Llama 2 13B
customizationsSupported: FINE_TUNING
-----
modelArn: arn:aws:bedrock:us-east-1::foundation-model/meta.llama2-70b-v1:0:4k
modelId: meta.llama2-70b-v1:0:4k
modelName: Llama 2 70B
customizationsSupported: FINE_TUNING


In [6]:
base_model_id = "meta.llama2-13b-v1:0:4k"

# 2. Test the base model

In [7]:
prompt = """
Summarize the simplest and most interesting part of the following conversation.

#Person1#: Hello. My name is John Sandals, and I've got a reservation.
#Person2#: May I see some identification, sir, please?
#Person1#: Sure. Here you are.
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals?
#Person1#: I sure do. How about American Express?
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA.
#Person1#: No American Express? Okay, here's my VISA.
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir?
#Person1#: Yeah, that'll be fine.
#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero.

Summary: 
"""

body = {
    "prompt": prompt,
    "temperature": 0.5,
    "top_p": 0.9,
    "max_gen_len": 512,
}

### Llama2 chat model 

In [7]:
response = bedrock_runtime.invoke_model(
    modelId="meta.llama2-13b-chat-v1", # compare to chat model
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

A man named John Sandals checks into a hotel and provides identification and a credit card. The hotel only takes MasterCard or VISA, so he uses his VISA card. He is given room 507, a nonsmoking room with a queen-size bed.


# 3. Prepare the dataset
https://huggingface.co/datasets/knkarthick/dialogsum

![](images/dataset_sm.png)

In [9]:
from datasets import load_dataset
dataset = load_dataset("knkarthick/dialogsum").remove_columns(["id", "topic"])
dataset

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 500
    })
    test: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 1500
    })
})

In [10]:
pprint(dataset["train"][0])

{'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here "
             'today?\n'
             '#Person2#: I found it would be a good idea to get a check-up.\n'
             "#Person1#: Yes, well, you haven't had one for 5 years. You "
             'should have one every year.\n'
             '#Person2#: I know. I figure as long as there is nothing wrong, '
             'why go see the doctor?\n'
             '#Person1#: Well, the best way to avoid serious illnesses is to '
             'find out about them early. So try to come at least once a year '
             'for your own good.\n'
             '#Person2#: Ok.\n'
             '#Person1#: Let me see here. Your eyes and ears look fine. Take a '
             'deep breath, please. Do you smoke, Mr. Smith?\n'
             '#Person2#: Yes.\n'
             '#Person1#: Smoking is the leading cause of lung cancer and heart '
             'disease, you know. You really should quit.\n'
             "#Person2#: I've tried

In [11]:
def wrap_instruction_fn(example):
    prompt = 'Summarize the simplest and most interesting part of the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    example["instruction"] = prompt + example["dialogue"] + end_prompt
    return example

In [12]:
dataset['train']\
  .select(range(1000))\
  .select_columns(['dialogue', 'summary'])\
  .map(wrap_instruction_fn)\
  .remove_columns(['dialogue'])\
  .rename_column('instruction', 'prompt')\
  .rename_column('summary', 'completion')\
  .to_json('./train-summarization.jsonl', index=False)

In [13]:
df = pd.read_json("./train-summarization.jsonl", lines=True)
df

,completion,prompt
0,"Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking.","Summarize the simplest and most interesting part of the following conversation.\n\n#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n..."
1,Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.,"Summarize the simplest and most interesting part of the following conversation.\n\n#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.\n\nSummary:"
2,#Person1#'s looking for a set of keys and asks for #Person2#'s help to find them.,"Summarize the simplest and most interesting part of the following conversation.\n\n#Person1#: Excuse me, did you see a set of keys?\n#Person2#: What kind of keys?\n#Person1#: Five keys and a small foot ornament.\n#Person2#: What a shame! I didn't see them.\n#Person1#: Well, can you help me look for it? That's my first time here.\n#Person2#: Sure. It's my pleasure. I'd like to help you look for the missing keys.\n#Person1#: It's very kind of you.\n#Person2#: It's not a big deal.Hey, I found them.\n#Person1#: Oh, thank God! I don't know how to thank you, guys.\n#Person2#: You're welcome.\n\nSummary:"
3,#Person1#'s angry because #Person2# didn't tell #Person1# that #Person2# had a girlfriend and would marry her.,"Summarize the simplest and most interesting part of the following conversation.\n\n#Person1#: Why didn't you tell me you had a girlfriend?\n#Person2#: Sorry, I thought you knew.\n#Person1#: But you should tell me you were in love with her.\n#Person2#: Didn't I?\n#Person1#: You know you didn't.\n#Person2#: Well, I am telling you now.\n#Person1#: Yes, but you might have told me before.\n#Person2#: I didn't think you would be interested.\n#Person1#: You can't be serious. How dare you not tell me you are going to marry her?\n#Person2#: Sorry, I didn't think it mattered.\n#Person1#: Oh, you men! You are all the same.\n\nSummary:"
4,Malik invites Nikki to dance. Nikki agrees if Malik doesn't mind getting his feet stepped on.,"Summarize the simplest and most interesting part of the following conversation.\n\n#Person1#: Watsup, ladies! Y'll looking'fine tonight. May I have this dance?\n#Person2#: He's cute! He looks like Tiger Woods! But, I can't dance. . .\n#Person1#: It's all good. I'll sh

# 4. Upload our dataset to S3

In [14]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session_bucket = sess.default_bucket()

s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train-summarization.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [15]:
!aws s3 cp ./train-summarization.jsonl $s3_location

upload: ./train-summarization.jsonl to s3://sagemaker-us-east-1-079002598131/bedrock/finetuning/train-summarization.jsonl


# 5. Customize the model with fine-tuning

In [16]:
timestamp = int(time.time())

job_name = "llama2-{}".format(timestamp)

custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-llama2-1701126112'

In [17]:
bedrock.create_model_customization_job(
    customizationType="FINE_TUNING",
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "1",
        "batchSize": "1",
        "learningRate": "0.000005"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

In [18]:
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(600)
    status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

Completed


In [19]:
custom_model_arn = bedrock.get_custom_model(modelIdentifier=custom_model_name)['modelArn']

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/bedrock/home?region={}#/custom-models/item?arn={}">Custom Model</a></b>'.format(
            region, custom_model_arn
        )
    )
)

# 6. Provision the custom model for inference

In [20]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)

base_model_arn = bedrock.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']

bedrock.create_provisioned_model_throughput(
    modelUnits = 1,
    provisionedModelName = provisioned_model_name,
    modelId = base_model_arn
) 

In [21]:
deployment_status = bedrock.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    print(deployment_status)
    time.sleep(120)
    deployment_status = bedrock.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

Creating
Creating
Creating
Creating
Creating
InService


In [22]:
provisioned_model_arn = bedrock.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/bedrock/home?region={}#/provisioned-throughput/details?arn={}">Custom Model Inference</a></b>'.format(
            region, provisioned_model_arn
        )
    )
)

# 7. Test the custom model

In [23]:
prompt = """
Summarize the simplest and most interesting part of the following conversation.

#Person1#: Hello. My name is John Sandals, and I've got a reservation.
#Person2#: May I see some identification, sir, please?
#Person1#: Sure. Here you are.
#Person2#: Thank you so much. Have you got a credit card, Mr. Sandals?
#Person1#: I sure do. How about American Express?
#Person2#: Unfortunately, at the present time we take only MasterCard or VISA.
#Person1#: No American Express? Okay, here's my VISA.
#Person2#: Thank you, sir. You'll be in room 507, nonsmoking, with a queen-size bed. Do you approve, sir?
#Person1#: Yeah, that'll be fine.\\n#Person2#: That's great. This is your key, sir. If you need anything at all, anytime, just dial zero.

Summary: 
"""

body = {
    "prompt": prompt,
    "temperature": 0.5,
    "top_p": 0.9,
    "max_gen_len": 512,
}

### Llama2 chat model 

In [24]:
response = bedrock_runtime.invoke_model(
    modelId="meta.llama2-13b-chat-v1", # compare to chat model
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

A man named John Sandals checks into a hotel and provides identification and a credit card. The hotel only takes MasterCard or VISA, so he uses his VISA card. He is given room 507, a nonsmoking room with a queen-size bed.


### Our custom fine-tuned model

In [25]:
response = bedrock_runtime.invoke_model(
    modelId=provisioned_model_arn, # custom fine-tuned model
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

John Sandals checks in the hotel with VISA and is assigned room 507, nonsmoking, with a queen-size bed.


# 8. Delete provisioned model to save cost

In [26]:
bedrock.delete_provisioned_model_throughput(
    provisionedModelId = provisioned_model_name
)

# GitHub repo
https://github.com/generative-ai-on-aws/generative-ai-on-aws

![](images/github.png)